## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [4]:
# Create a list of at least 10 column names to use as X data

columns = [ col for col in attrition_df.columns if col not in ['Attrition','Department']]

print(len(columns))

select_columns = ['YearsSinceLastPromotion','YearsInCurrentRole','YearsAtCompany','WorkLifeBalance','StockOptionLevel','PercentSalaryHike','PerformanceRating','JobSatisfaction','JobRole','HourlyRate','OverTime','DistanceFromHome']

# Create X_df using your selected columns
X_df = attrition_df[select_columns]


# Show the data types for X_df
X_df.info()


25
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   YearsSinceLastPromotion  1470 non-null   int64 
 1   YearsInCurrentRole       1470 non-null   int64 
 2   YearsAtCompany           1470 non-null   int64 
 3   WorkLifeBalance          1470 non-null   int64 
 4   StockOptionLevel         1470 non-null   int64 
 5   PercentSalaryHike        1470 non-null   int64 
 6   PerformanceRating        1470 non-null   int64 
 7   JobSatisfaction          1470 non-null   int64 
 8   JobRole                  1470 non-null   object
 9   HourlyRate               1470 non-null   int64 
 10  OverTime                 1470 non-null   object
 11  DistanceFromHome         1470 non-null   int64 
dtypes: int64(10), object(2)
memory usage: 137.9+ KB


In [5]:
attrition_df['Department'].value_counts() #,'Department']

Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64

In [6]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df)

In [7]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

encode_overtime = OrdinalEncoder(categories=[['No', 'Yes']], handle_unknown='use_encoded_value', unknown_value=-1)

# Train the encoder
encode_overtime.fit(X_train['OverTime'].values.reshape(-1, 1))


OrdinalEncoder(categories=[['No', 'Yes']], handle_unknown='use_encoded_value',
               unknown_value=-1)

In [8]:
encode_JobRole = OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False)

# Train the encoder
encode_JobRole.fit(X_train['JobRole'].values.reshape(-1, 1))

OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False)

In [9]:

def encode_categorical(X_data):
    X_data_numeric = X_data.select_dtypes(include='number').reset_index()
    
    jobrole_encoded_df = pd.DataFrame(encode_JobRole.transform(X_data['JobRole'].values.reshape(-1, 1)), columns=encode_JobRole.get_feature_names_out())
    
    dfs = [X_data_numeric, jobrole_encoded_df]
    X_data_encoded = pd.concat(dfs, axis=1)

    
    X_data_encoded['OverTime'] = encode_overtime.transform(X_data['OverTime'].values.reshape(-1, 1))

    return X_data_encoded


In [10]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train_encoded = encode_categorical(X_train)
X_test_encoded = encode_categorical(X_test)

In [11]:
# Create a StandardScaler

ssScaler = StandardScaler()

# Fit the StandardScaler to the training data
ssScaler.fit(X_train_encoded)

# Scale the training and testing data

X_train_transform = ssScaler.transform(X_train_encoded)
X_test_transform = ssScaler.transform(X_test_encoded)


X_train_encoded['OverTime'].value_counts()

OverTime
0.0    788
1.0    314
Name: count, dtype: int64

In [12]:
# Create a OneHotEncoder for the Department column

encode_Dept = OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False)

encode_Dept.fit(y_train['Department'].values.reshape(-1, 1))


def encode_dept_categorical(Y_data):
    #Y_data_rest = Y_data[ [col for col in Y_data.columns if col != 'Department'] ].reset_index(drop=True)
    
    dept_encoded_df = pd.DataFrame(encode_Dept.transform(Y_data['Department'].values.reshape(-1, 1)), columns=encode_Dept.get_feature_names_out())
    
    #dfs = [Y_data_rest, dept_encoded_df]
    #Y_data_encoded = pd.concat(dfs, axis=1)
    

    return dept_encoded_df


In [13]:

# Create two new variables by applying the encoder
# to the training and testing data

y_train_dept_encode = encode_dept_categorical(y_train)
y_test_dept_encode = encode_dept_categorical(y_test)

print(y_test_dept_encode.columns)



Index(['x0_Research & Development', 'x0_Sales'], dtype='object')


In [14]:
# Create a OneHotEncoder for the Attrition column

# Fit the encoder to the training data
encode_attrition = OrdinalEncoder(categories=[['No', 'Yes']], handle_unknown='use_encoded_value', unknown_value=-1)

# Train the encoder
encode_attrition.fit(y_train['Attrition'].values.reshape(-1, 1))

def encode_attr_categorical(Y_data):
    #Y_data_rest = Y_data[ [col for col in Y_data.columns if col != 'Attrition'] ].reset_index(drop=True)
    
    attr_encoded_df = pd.DataFrame(encode_attrition.transform(Y_data['Attrition'].values.reshape(-1, 1)), columns=encode_attrition.get_feature_names_out())
    
    #dfs = [Y_data_rest, attr_encoded_df]
    #Y_data_encoded = pd.concat(dfs, axis=1)
    
    return attr_encoded_df


In [15]:
# Create two new variables by applying the encoder
# to the training and testing data

y_train_attrition_encode = encode_attr_categorical(y_train)
y_test_attrition_encode = encode_attr_categorical(y_test)

In [16]:
y_train_attrition_encode.columns = ['Attrition']
y_test_attrition_encode.columns = ['Attrition']

y_train_dept_encode.columns = ['Research & Development', 'Sales']
y_test_dept_encode.columns = ['Research & Development', 'Sales']

In [17]:
X_train_transform.shape

(1102, 20)

## Create, Compile, and Train the Model

In [18]:
# Find the number of columns in the X training data
nosOfColumns = X_train_transform.shape[1]

# Create the input layer
inputLayer = layers.Input(shape=(nosOfColumns,), name='inputLayer')

# Create at least two shared layers
layer1 = layers.Dense(units=2*nosOfColumns+1, activation="relu", name='layer1')(inputLayer)
layer2 = layers.Dense(units=2*nosOfColumns-5, activation="tanh", name='layer2')(layer1)


In [19]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden  = layers.Dense(units=2*nosOfColumns - 5, activation="relu" , name ='department_hidden') (layer2)


# Create the output layer
department_output  = layers.Dense(units=2, activation="softmax", name='department_output')(department_hidden)


In [20]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(units=2*nosOfColumns, activation="relu", name='attrition_hidden')(layer2)

# Create the output layer
attrition_output = layers.Dense(units=1, activation="sigmoid", name='attrition_output') (attrition_hidden)


In [21]:
# Create the model


model = Model(inputs=inputLayer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'categorical_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})


# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ inputLayer          │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer1 (Dense)      │ (None, 41)        │        861 │ inputLayer[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer2 (Dense)      │ (None, 35)        │      1,470 │ layer1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_hidden   │ (None, 35)        │      1,260 │ layer2[0][0]      │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden    │ (None, 40)        │      1,440 │ layer2[0][0]      │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 2)         │         72 │ department_hidde… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 1)         │         41 │ attrition_hidden… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,144 (20.09 KB)

 Trainable params: 5,144 (20.09 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# Train the model
model.fit(
    X_train_transform,
    {'attrition_output': y_train_attrition_encode, 'department_output': y_train_dept_encode},
    epochs=20,
    batch_size=50,
    validation_split=0.2
)


Epoch 1/20


C:\Users\deepa\AppData\Roaming\Python\Python312\site-packages\keras\src\losses\losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


14/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.7251 - attrition_output_loss: 0.0000e+00 - department_output_accuracy: 0.2707 - department_output_loss: 0.8053 - loss: 0.8053

C:\Users\deepa\AppData\Roaming\Python\Python312\site-packages\keras\src\losses\losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - attrition_output_accuracy: 0.7434 - attrition_output_loss: 0.0000e+00 - department_output_accuracy: 0.2964 - department_output_loss: 0.7831 - loss: 0.7834 - val_attrition_output_accuracy: 0.8009 - val_attrition_output_loss: 0.0000e+00 - val_department_output_accuracy: 0.6199 - val_department_output_loss: 0.6100 - val_loss: 0.6080
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - attrition_output_accuracy: 0.8341 - attrition_output_loss: 0.0000e+00 - department_output_accuracy: 0.7337 - department_output_loss: 0.5786 - loss: 0.5788 - val_attrition_output_accuracy: 0.8009 - val_attrition_output_loss: 0.0000e+00 - val_department_output_accuracy: 0.8688 - val_department_output_loss: 0.4916 - val_loss: 0.4930
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - attrition_output_accuracy: 0.8299 - attrition_output_loss: 0.0000e+00 - department_output_accuracy: 0.8483 - department_output_loss: 0.4690 - loss: 0.4691 - val_attrition_output_accuracy: 

In [23]:
# Evaluate the model with the testing data
model_loss, department_output_loss, attrition_output_loss, attrition_model_accuracy, dept_model_accuracy= model.evaluate(X_test_transform, {'attrition_output': y_test_attrition_encode, 'department_output': y_test_dept_encode}, verbose=2)


12/12 - 0s - 8ms/step - attrition_output_accuracy: 0.8451 - attrition_output_loss: 0.0000e+00 - department_output_accuracy: 0.7038 - department_output_loss: 0.5911 - loss: 0.5921


In [24]:
# Print the accuracy for both department and attrition
print(attrition_model_accuracy)
print(dept_model_accuracy)

0.845108687877655
0.7038043737411499


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. The accuracy is good metric to evaluate the model's performance. It measures the proportion of correctly classified instances out of all instances. The use case is prediction of attrition and dept which is not crucial. F1 score can be used to evaluate the model's performance when false negative and false positive are crucial for usecase.
2. For attrition output, its binary classification yes or no, so sigmoid is used as its for binary classification. For dept output, its multiple classes involved hence softmax is used. 
3. Its difficult to predict the department which is suitable for the employee. The model is trained on the data which is available and there should more data collected for department like what skills, nos of employee in dept and other details to identify the suitable department for the employee.